# Taller 1 Buscador

Se procesan inicialmente todos los archivos que tienen las noticias.

In [1]:
import time
import re
import math

from bs4 import BeautifulSoup   

start_time = time.time()

# Abrir el archivo inicial y tomar el documento html.

archivo = open("archivosinicial/reut2-000.sgm", "r")
soup1  = BeautifulSoup(archivo, 'html.parser')
archivo.close()

documentos = []
documentos = soup1.find_all('reuters')

fileInitial = open("archivoprocesado/reut2-000.txt", "w")
fileRaw = open("archivoprocesado/raw.txt", "w")

for i in range(len(documentos)):
        try:          
            cadena = documentos[i].title.string.replace('\n',' ')+" "+documentos[i].body.string.replace('\n',' ')
            cadena = cadena.lower()
            cadena = cadena.replace(' reuter','')           
            fileInitial.write(cadena+"\n") 
            cadena=documentos[i].title.string + "@@,"+ documentos[i].body.string +"@@;" 
            fileRaw.write(cadena+"\n") 
        except:
            pass   

fileInitial.close()
fileRaw.close()

words = []
stopwords = open("archivosinicial/stopwords.txt", "r")
dato = stopwords.readline()
words = dato.split(';')
stopwords.close()


Se define la funcion en la cual se aplican expresiones regulares para limpiar los documentos recibidos.

In [2]:
def processDocument(documentos,archivoProcesado):
    for cadena in documentos.readlines():
        try:                      
            cadena = re.sub(r'<.*>|[0-9]|[,*$]|[.*$]|[-*$]|[(.*)$]|[/*$]|["*$]|[\'][a-z|\W]|[+*$]|[:*$]'," ",cadena)        
            for j in range(len(words)): 
                cadena = re.sub(" " + words[j]+" "," ",cadena)
                
            archivoProcesado.write(cadena)                   
        except:
            pass        
    

archivoProcesado = open("archivoprocesado/fileDocumentos.txt", "w")
documentos =  open("archivosinicial/reut2-000.txt", "r")
processDocument(documentos,archivoProcesado)
archivoProcesado.close()
documentos.close()


Crear el diccionario de los documentos recibidos.
Se define un patron para hacer split en lo que no sea caracteres alfanumericos y se obtiene el diccionario de palabras y se aplica la funcion set para quitar duplicados.

In [3]:
import re
def getDictionary(fileDoc,fileDic):
    pattern = re.compile(r'\W+')

    fileWords = open("archivoprocesado/"+fileDoc, "r")
    dictionary = pattern.split(fileWords.read())
    
    dictionary = sorted(list(set(dictionary)))
   
    dictionary.remove("")
    fileWords.close()
    
    dicString = str(dictionary).lstrip("[").rstrip("]").replace(", ",",")
    dicArray  = dicString.split(",")
    dicFinal = ""
    
    for i in range(len(dicArray)):
        dicFinal = dicFinal+","+dicArray[i].lstrip("'").rstrip("'")
    
    
    #Guardar el diccionar en un archivo.
    fileDictionary = open("archivoprocesado/"+fileDic, "w")
    fileDictionary.write(dicFinal.lstrip(","))
    fileDictionary.close()
    
getDictionary("fileDocumentos.txt","dictionary.txt")

Funcion que crea la matrizTF y la guarda en un archivo, cuenta la cantidad de palabras en cada documento para crear el vector con los denominadores para obtener la matrizTFIDF.

In [4]:
import re

def setMatrizTF(fileMatriz,fileDic,fileDoc):
    
    dictionary  = open('archivoprocesado/'+fileDic).read()    
    dictionary  = dictionary.split(",")
    
    #Crear el Vector Space Model
    vectorSpace = open("archivoprocesado/"+fileMatriz, "w")
    fileFinally = open("archivoprocesado/"+fileDoc, "r")

    w=len(dictionary)
    h=len(fileFinally.readlines())

    n=0

    denominador=[0 for x in range(w)]

    linea=""

    fileFinally = open("archivoprocesado/"+fileDoc, "r")
    for lines in fileFinally.readlines(): 
        lineaAux=""
        for k in range(len(dictionary)):                                                 
            patron = re.compile(r''+dictionary[k]+'')        
            count = len(patron.findall(lines))

            lineaAux = lineaAux+","+str(count)

            if count > 0:            
                denominador[k]=denominador[k]+1
            else:
                denominador[k]=denominador[k]+0

        lineaAux = lineaAux.lstrip(",")
        linea = linea+lineaAux +"\n"
        n=n+1    

    linea = linea.rstrip("\n")
    vectorSpace.write(linea)
    vectorSpace.close()
    fileFinally.close()  
    
    fileDen = open("archivoprocesado/denominador.txt", "w")
    fileDen.write(str(denominador).lstrip("[").rstrip("]").replace(", ",","))
    fileDen.close()
    
setMatrizTF("matriz_tf.txt","dictionary.txt","fileDocumentos.txt")  

Funcion que crea la matrizTFIDF, aplicando la formula

In [5]:
def SimilitudCoseno(VectorArchivo, vectorBusqueda):
    x = np.array(VectorArchivo)
    y = np.array(vectorBusqueda)
    dot = np.dot(x,y)
    x_modulus = np.sqrt((x*x).sum())
    y_modulus = np.sqrt((y*y).sum())
    similitud=0
    if(x_modulus != 0 and y_modulus != 0 ):
        similitud = dot / x_modulus / y_modulus
    return similitud

In [6]:
# Obtener matriz TF-IDF
import math
import time

InvertedIndex = []
TFIDFDb=[]
def getMatrizTFIDF():
    den= open("archivoprocesado/denominador.txt", "r").read()
    denominador = den.split(",")

    matrizTF = open('archivoprocesado/matriz_tf.txt').read()
    matrizTF = [item.split() for item in matrizTF.split('\n')]

    w=len(denominador)
    h=len(matrizTF)

    #Creamos matriz TF-IDF
    fileMatriz= open("archivoprocesado/matriz_tfidf.txt", "w")

    doc = []
    
    logTerm=[]
    
    for i in range(len(matrizTF)):
        fila = matrizTF[i]  
        doc  = str(fila).lstrip("['").rstrip("']").split(",")
        nq = []
        
        for j in range(len(doc)): 

            if i==0:
                InvertedIndex.append([])
                logTerm.append([])
                logTerm[j]=math.log(h/int(denominador[j]))
                tfidf= int(doc[j])*logTerm[j]
                if(tfidf>0):
                    nq.append([j,tfidf])
            else:
                tfidf = int(doc[j])*logTerm[j]
                if(tfidf>0):
                    nq.append([j,tfidf])
            if(tfidf>0):
                InvertedIndex[j].append(i)
                
        TFIDFDb.append( {"id":i,"tf_idf":nq} )
   
    print(" ------ Tiempo de ejecucion %s -----------" % (time.time()-start_time))   


    #Guardar matriz TF - IDF            

       

getMatrizTFIDF()    

 ------ Tiempo de ejecucion 0.26602602005004883 -----------


Funciones para tomar la informacion ingresada en la caja de texto, crear el vector query y hacer similitud de coseno con la matriz tf-idf

In [7]:
from tkinter import *
import re
import numpy as np
from IPython.core.display import display, HTML
from IPython.display import clear_output
from IPython.display import Markdown, display

#Generar la ventana que recibe el texto del buscador
vent = Tk()

a = StringVar()

def vectorSearch():
    pattern = re.compile(r'\W+')
    file  = open('archivoprocesado/fileSearch.txt').read()    
    wordsSearch  = pattern.split(file)     
     
    dictionary  = open('archivoprocesado/dictionary.txt').read()    
    dictionary  = dictionary.split(",")
    
    w=len(dictionary)
    vector = []
    vector = [0 for x in range(w)]
    IdWords=[]
    IdDocuments=[]
    resultadosBusqueda=[]
    for i in range(len(wordsSearch)):
        try:
            
            ind = dictionary.index(wordsSearch[i])
            IdWords.append(ind)
            if(vector[ind]>0):
                vector[ind] = vector[ind]+1
            else:    
                vector[ind] = 1
        except:
            pass
    
    if(len(IdWords)>0):
        #Se busca con el indice invertido los documentos que tienen las palabras a buscar
        for countWord in range(0,len(IdWords)):
            for IdDocument in InvertedIndex[IdWords[countWord]]:
                if(IdDocument not in IdDocuments):
                    IdDocuments.append(IdDocument)
    
        countArchivos=0   
        #Se recorre cada documento
        for IdDocument in IdDocuments:
                tf_idf =  [x for x in TFIDFDb if x['id'] == IdDocument][0]['tf_idf']
                #armo la matriz TF-IDF
                VectorArchivo = np.zeros(w)
                for x in range(len(tf_idf)):
                    VectorArchivo[tf_idf[x][0]]=tf_idf[x][1]
                
                resultadosBusqueda.append([])
                #Calcula la similitud coseno de cada documento con el vector de busqueda
                similitud = SimilitudCoseno(VectorArchivo, vector)

                resultadosBusqueda[countArchivos].append(IdDocument)
                resultadosBusqueda[countArchivos].append(similitud)
                countArchivos+=1
        resultadosBusqueda = sorted(resultadosBusqueda, key=lambda a_entry: a_entry[1],reverse=True) 

        #Se recuperan los documentos originales sin procesar
        documentosRaw=open("archivoprocesado/raw.txt", "r").read().split("@@;")
        clear_output()
        #con los los resultados de la busqueda visualiza los documentos encontrados
        for resultado in resultadosBusqueda:
            print(resultado)
            display(HTML('<h1>'+documentosRaw[resultado[0]].split("@@,")[0]+'</h1>'))
            display(HTML('<p>'+documentosRaw[resultado[0]].split("@@,")[1]+'</p>'))
    
    
    # vector tiene el query listo para aplicar similtud de coseno
    #Leer la matriztf-idf para hacer similutd de coseno con el vector de query
    
    

def valor():
    a = entrada.get()
    fileWords = open("archivoprocesado/buscador.txt", "w")
    a = a.lower()
    fileWords.write(a)
    fileWords.close()
    
    #Procesar las palabra ingresadas.
    documentos=open("archivoprocesado/buscador.txt", "r")
    archivoProcesado = open("archivoprocesado/fileSearch.txt", "w")
    processDocument(documentos,archivoProcesado)
    documentos.close()
    archivoProcesado.close()
    
    vectorSearch()


entrada = Entry(vent, width=30)
entrada.grid(row=0, column=0)

boton = Button(vent, text="Buscar", command=valor)
boton.grid(row=1, column=0)

vent.mainloop()


print(" ------ Tiempo de ejecucion %s -----------" % (time.time()-start_time))     

[2, 0.1468929125857383]


 ------ Tiempo de ejecucion 102.14594101905823 -----------
